In [3]:
import requests
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('copenhagen_postalcodes.csv', delimiter='\t', error_bad_lines=False)

In [5]:
data.head()

,Postcode,City/town name,Street,Firm,Region,Country
0,1050,Kobenhavn K,Kongens Nytorv,NaN,False,1.0
1,1051,Kobenhavn K,Nyhavn,NaN,False,1.0
2,1052,Kobenhavn K,Herluf Trolles Gade,NaN,False,1.0
3,1053,Kobenhavn K,Cort Adelers Gade,NaN,False,1.0
4,1054,Kobenhavn K,Peder Skrams Gade,NaN,False,1.0


In [6]:
data.drop(['Firm'], axis=1, inplace=True)
data.drop(['Region'], axis=1, inplace=True)
data.drop(['Country'], axis=1, inplace=True)

In [7]:
data.head()

,Postcode,City/town name,Street
0,1050,Kobenhavn K,Kongens Nytorv
1,1051,Kobenhavn K,Nyhavn
2,1052,Kobenhavn K,Herluf Trolles Gade
3,1053,Kobenhavn K,Cort Adelers Gade
4,1054,Kobenhavn K,Peder Skrams Gade


In [8]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [9]:
!pip install geocoder

In [10]:
import geocoder

In [11]:
# define the dataframe columns
column_names = ['Postcode','City/town name', 'Street', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

#empty dataframe
neighborhoods.head()

,Postcode,City/town name,Street,Latitude,Longitude


In [ ]:
lastvalue = data.shape[0]

n = 0

while(n<lastvalue) : 
    postal_code = data['Postcode'][n]
    city = data['City/town name'][n]
    street = data['Street'][n]
    #while(lat_lng_coords is None) :
    g = geocoder.arcgis('{}, Copenhagen, Denmark'.format(postal_code))
    lat_lng_coords = g.latlng
    neighborhood_lat = lat_lng_coords[0]
    neighborhood_lon = lat_lng_coords[1]
   # print(n, greg, lat_lng_coords, borough, neighborhood_name, neighborhood_lat, neighborhood_lon)
    
    neighborhoods = neighborhoods.append({'Postcode': postal_code,
                                          'City/town name': city,
                                          'Street': street,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon},ignore_index=True)
    
    n = n+1

In [ ]:
neighborhoods.head()

In [ ]:
neighborhoods.shape

In [ ]:
from geopy.geocoders import Nominatim
import folium

In [ ]:
address = 'Copenhagen, Denmark'

geolocator = Nominatim(user_agent="copenhagen_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Copenhagen are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Toronto using latitude and longitude values
map_copenhagen = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, city in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['City/town name']):
    
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        
        color='gray',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_copenhagen)  
    
map_copenhagen

In [ ]:
CLIENT_ID = 'Q0ITL1ZB5Z2J554VCKJTHJOYKXGHHHRISFOGDPJ4PFGE0FHG' # your Foursquare ID
CLIENT_SECRET = 'NWBZLOFJQ41XMF5V5FNXPSZSJGUNZALLSEOUFUTO3DWMV2IM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

In [ ]:
def getNearbyVenues_ari(names, postalcodes, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, postalCode, lat, lng in zip(names, postalcodes, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            postalCode,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                             'PostalCode',
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
copenhagen_venues = getNearbyVenues_ari(names=neighborhoods['City/town name'],
                                       
                                  postalcodes=neighborhoods['Postcode'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

In [ ]:
print(copenhagen_venues.shape)
copenhagen_venues.head()

In [ ]:
copenhagen_venues.to_csv(r'C:\Users\Ari_Greg\Documents\copenhagen_venues.csv', index =None, header=True)

In [ ]:
print('There are {} uniques categories.'.format(len(copenhagen_venues['Venue Category'].unique())))

grouped=copenhagen_venues.groupby('District').count()
sorted=grouped.sort_values(by='PostalCode', ascending=False)
sorted.head(50)

In [ ]:
# one hot encoding
copenhagen_onehot = pd.get_dummies(copenhagen_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
copenhagen_onehot['District'] = copenhagen_venues['District']
 

# move neighborhood column to the first column
fixed_columns = [copenhagen_onehot.columns[-1]] + list(copenhagen_onehot.columns[:-1])
copenhagen_onehot = copenhagen_onehot[fixed_columns]

pd.set_option('display.max_columns', None)

copenhagen_onehot.head()

In [ ]:
copenhagen_onehot.shape

In [ ]:
copenhagen_grouped.shape